In [1]:
import sys
sys.path.insert(0, '../ELINA/python_interface/')

import numpy as np
import re
import csv
from elina_box import *
from elina_interval import *
from elina_abstract0 import *
from elina_manager import *
from elina_dimension import *
from elina_scalar import *
from elina_interval import *
from elina_linexpr0 import *
from elina_lincons0 import *
import ctypes
from ctypes.util import find_library
from gurobipy import *
import time
from pprint import pprint
import copy

In [2]:
# Import for debugging in jupyter notebook
from IPython.core.debugger import set_trace #TODO remove at end.

In [3]:
libc = CDLL(find_library('c'))
cstdout = c_void_p.in_dll(libc, 'stdout')

In [4]:
class layers:
    def __init__(self):
        self.layertypes = []
        self.weights = []
        self.biases = []
        self.numlayer = 0
        self.ffn_counter = 0
        self.rank = []
        self.use_LP = []

In [5]:
def parse_bias(text):
    if len(text) < 1 or text[0] != '[':
        raise Exception("expected '['")
    if text[-1] != ']':
        raise Exception("expected ']'")
    v = np.array([*map(lambda x: np.double(x.strip()), text[1:-1].split(','))])
    #return v.reshape((v.size,1))
    return v

In [6]:
def parse_vector(text):
    if len(text) < 1 or text[0] != '[':
        raise Exception("expected '['")
    if text[-1] != ']':
        raise Exception("expected ']'")
    v = np.array([*map(lambda x: np.double(x.strip()), text[1:-1].split(','))])
    return v.reshape((v.size,1))
    #return v

In [7]:
def balanced_split(text):
    i = 0
    bal = 0
    start = 0
    result = []
    while i < len(text):
        if text[i] == '[':
            bal += 1
        elif text[i] == ']':
            bal -= 1
        elif text[i] == ',' and bal == 0:
            result.append(text[start:i])
            start = i+1
        i += 1
    if start < i:
        result.append(text[start:i])
    return result

In [8]:
def parse_matrix(text):
    i = 0
    if len(text) < 1 or text[0] != '[':
        raise Exception("expected '['")
    if text[-1] != ']':
        raise Exception("expected ']'")
    return np.array([*map(lambda x: parse_vector(x.strip()).flatten(), balanced_split(text[1:-1]))])

In [9]:
def parse_net(text):
    lines = [*filter(lambda x: len(x) != 0, text.split('\n'))]
    i = 0
    res = layers()
    while i < len(lines):
        if lines[i] in ['ReLU', 'Affine']:
            W = parse_matrix(lines[i+1])
            b = parse_bias(lines[i+2])
            res.layertypes.append(lines[i])
            res.weights.append(W)
            res.biases.append(b)
            res.numlayer+= 1
            res.rank.append(np.zeros((W.shape[0],1)))
            res.use_LP.append(np.full((W.shape[0],1), False))
            i += 3
        else:
            raise Exception('parse error: '+lines[i])
    return res

In [10]:
def parse_spec(text):
    text = text.replace("[", "")
    text = text.replace("]", "")
    with open('dummy', 'w') as my_file:
        my_file.write(text)
    data = np.genfromtxt('dummy', delimiter=',',dtype=np.double)
    low = copy.deepcopy(data[:,0])
    high = copy.deepcopy(data[:,1])
    return low,high

In [11]:
def get_perturbed_image(x, epsilon):
    image = x[1:len(x)]
    num_pixels = len(image)
    LB_N0 = image - epsilon
    UB_N0 = image + epsilon
     
    for i in range(num_pixels):
        if(LB_N0[i] < 0):
            LB_N0[i] = 0
        if(UB_N0[i] > 1):
            UB_N0[i] = 1
    return LB_N0, UB_N0

In [12]:
def generate_linexpr0(weights, bias, size):
    linexpr0 = elina_linexpr0_alloc(ElinaLinexprDiscr.ELINA_LINEXPR_DENSE, size)
    cst = pointer(linexpr0.contents.cst)
    elina_scalar_set_double(cst.contents.val.scalar, bias)
    for i in range(size):
        elina_linexpr0_set_coeff_scalar_double(linexpr0,i,weights[i])
    return linexpr0

In [13]:
def analyze(nn, LB_N0, UB_N0, label):   
    num_pixels = len(LB_N0)
    nn.ffn_counter = 0
    numlayer = nn.numlayer 
    man = elina_box_manager_alloc()
    itv = elina_interval_array_alloc(num_pixels)
    for i in range(num_pixels):
        elina_interval_set_double(itv[i],LB_N0[i],UB_N0[i])

    ## construct input abstraction
    element = elina_abstract0_of_box(man, 0, num_pixels, itv)
    elina_interval_array_free(itv,num_pixels)
    for layerno in range(numlayer):
        if(nn.layertypes[layerno] in ['ReLU', 'Affine']):
            weights = nn.weights[nn.ffn_counter]
            biases = nn.biases[nn.ffn_counter]
            dims = elina_abstract0_dimension(man,element)
            num_in_pixels = dims.intdim + dims.realdim
            num_out_pixels = len(weights)

            dimadd = elina_dimchange_alloc(0,num_out_pixels)    
            for i in range(num_out_pixels):
                dimadd.contents.dim[i] = num_in_pixels
            elina_abstract0_add_dimensions(man, True, element, dimadd, False)
            elina_dimchange_free(dimadd)
            np.ascontiguousarray(weights, dtype=np.double)
            np.ascontiguousarray(biases, dtype=np.double)
            var = num_in_pixels
            # handle affine layer
            for i in range(num_out_pixels):
                tdim= ElinaDim(var)
                linexpr0 = generate_linexpr0(weights[i],biases[i],num_in_pixels)
                element = elina_abstract0_assign_linexpr_array(man, True, element, tdim, linexpr0, 1, None)
                var+=1
            dimrem = elina_dimchange_alloc(0,num_in_pixels)
            for i in range(num_in_pixels):
                dimrem.contents.dim[i] = i
            elina_abstract0_remove_dimensions(man, True, element, dimrem)
            elina_dimchange_free(dimrem)
            # handle ReLU layer 
            if(nn.layertypes[layerno]=='ReLU'):
                element = relu_box_layerwise(man,True,element,0, num_out_pixels)
            nn.ffn_counter+=1 

        else:
            print(' net type not supported')
   
    dims = elina_abstract0_dimension(man,element)
    output_size = dims.intdim + dims.realdim
    # get bounds for each output neuron
    bounds = elina_abstract0_to_box(man,element)

           
    # if epsilon is zero, try to classify else verify robustness 
    
    verified_flag = True
    predicted_label = 0
    if(LB_N0[0]==UB_N0[0]):
        for i in range(output_size):
            inf = bounds[i].contents.inf.contents.val.dbl
            flag = True
            for j in range(output_size):
                if(j!=i):
                    sup = bounds[j].contents.sup.contents.val.dbl
                    if(inf<=sup):
                        flag = False
                        break
            if(flag):
                predicted_label = i
                break    
    else:
        inf = bounds[label].contents.inf.contents.val.dbl
        for j in range(output_size):
            if(j!=label):
                sup = bounds[j].contents.sup.contents.val.dbl
                if(inf<=sup):
                    predicted_label = label
                    verified_flag = False
                    break

    elina_interval_array_free(bounds,output_size)
    elina_abstract0_free(man,element)
    elina_manager_free(man)        
    return predicted_label, verified_flag

## Define operations on abstract domain using Box approximations

In [14]:
def get_relu_hidden_bounds_using_box(man, weights, biases, input_LB, input_UB, num_in_pixels, verbose=False):
    '''
    This function calculates the bounds of a ReLU operation followed by a hidden layer. 
    INPUT:
        - man: pointer to elina manager
        - weights: weights of the hidden layer
        - biases: biases of the hidden layer
        - input_LB: lower bound of the inputs to the ReLU
        - input_UB: upper bound of the inputs to the ReLU
        - num_in_pixels: number of inputs to ReLU
    
    OUTPUT:
        - output_LB: lower bound of the outputs from hidden layer
        - output_UB: upper bound of the outputs from hidden layer
        - num_out_pixels: number of outputs of hidden layer
    '''
    itv = elina_interval_array_alloc(num_in_pixels)

    ## Populate the interval
    for i in range(num_in_pixels):
        elina_interval_set_double(itv[i], input_LB[i], input_UB[i])

    ## construct input abstraction
    element = elina_abstract0_of_box(man, 0, num_in_pixels, itv)
    elina_interval_array_free(itv, num_in_pixels)
    
    # ------------------------------------------------------------------
    # Handle ReLU Layer
    # ------------------------------------------------------------------

    element = relu_box_layerwise(man, True, element,0, num_in_pixels)
    
    # ------------------------------------------------------------------
    # Handle Affine Layer
    # ------------------------------------------------------------------

    # calculate number of outputs
    num_out_pixels = len(weights)
    
    if verbose:
        print("[Network] Input pixels: " + str(num_in_pixels))
        print("[Network] Shape of weights: " + str(np.shape(weights)))
        print("[Network] Shape of biases: " + str(np.shape(biases)))
        print("[Network] Out pixels: " + str(num_out_pixels))

    # Create number of neurons in the layer and populate it
    # with the number of inputs to each neuron in the layer
    dimadd = elina_dimchange_alloc(0, num_out_pixels)    
    for i in range(num_out_pixels):
        dimadd.contents.dim[i] = num_in_pixels

    # Add dimensions to an ElinaAbstract0 pointer i.e. element
    elina_abstract0_add_dimensions(man, True, element, dimadd, False)
    elina_dimchange_free(dimadd)

    # Create the linear expression associated each neuron
    var = num_in_pixels
    for i in range(num_out_pixels):
        tdim = ElinaDim(var)
        linexpr0 = generate_linexpr0(weights[i], biases[i], num_in_pixels)
        # Parallel assignment of several dimensions of an ElinaAbstract0 by using an ElinaLinexpr0Array
        element = elina_abstract0_assign_linexpr_array(man, True, element, tdim, linexpr0, 1, None)
        var += 1

    # Pointer to which semantics we want to follow.
    dimrem = elina_dimchange_alloc(0, num_in_pixels)
    for i in range(num_in_pixels):
        dimrem.contents.dim[i] = i
        
    # Remove dimensions from an ElinaAbstract0
    elina_abstract0_remove_dimensions(man, True, element, dimrem)
    elina_dimchange_free(dimrem)
    
    # get bounds for each output neuron
    bounds = elina_abstract0_to_box(man,element)
    
    output_LB = np.zeros((num_out_pixels, 1), float)
    output_UB = np.zeros((num_out_pixels, 1), float)
    for j in range(num_out_pixels):
        output_LB[j] = bounds[j].contents.inf.contents.val.dbl
        output_UB[j] = bounds[j].contents.sup.contents.val.dbl
    
    # free out the memory allocations
    elina_interval_array_free(bounds, num_out_pixels)
    elina_abstract0_free(man, element)
    
    return output_LB, output_UB, num_out_pixels

In [15]:
def get_relu_bounds_using_box(man, input_LB, input_UB, num_in_pixels):
    '''
    This function calculates the bounds of a ReLU operation. 
    INPUT:
        - man: pointer to elina manager
        - input_LB: lower bound of the inputs to the ReLU
        - input_UB: upper bound of the inputs to the ReLU
        - num_in_pixels: number of inputs to ReLU
    
    OUTPUT:
        - output_LB: lower bound of the outputs from ReLU layer
        - output_UB: upper bound of the outputs from ReLU layer
        - num_out_pixels: number of outputs of ReLI layer
    '''
    itv = elina_interval_array_alloc(num_in_pixels)

    ## Populate the interval
    for i in range(num_in_pixels):
        elina_interval_set_double(itv[i], input_LB[i], input_UB[i])

    ## construct input abstraction
    element = elina_abstract0_of_box(man, 0, num_in_pixels, itv)
    elina_interval_array_free(itv, num_in_pixels)
    
    # ------------------------------------------------------------------
    # Handle ReLU Layer
    # ------------------------------------------------------------------
    num_out_pixels = num_in_pixels
    
    element = relu_box_layerwise(man, True, element,0, num_in_pixels)
    
    # get bounds for each output neuron
    bounds = elina_abstract0_to_box(man,element)
    
    # get bounds for each output neuron
    bounds = elina_abstract0_to_box(man,element)
    
    output_LB = np.zeros((num_out_pixels, 1), float)
    output_UB = np.zeros((num_out_pixels, 1), float)
    for j in range(num_out_pixels):
        output_LB[j] = bounds[j].contents.inf.contents.val.dbl
        output_UB[j] = bounds[j].contents.sup.contents.val.dbl
    
    # free out the memory allocations
    elina_interval_array_free(bounds, num_out_pixels)
    elina_abstract0_free(man, element)
    
    return output_LB, output_UB, num_out_pixels

In [16]:
def get_hidden_bounds_using_box(man, weights, biases, input_LB, input_UB, num_in_pixels, verbose=False):
    '''
    This function calculates the bounds of a ReLU operation followed by a hidden layer. 
    INPUT:
        - man: pointer to elina manager
        - weights: weights of the hidden layer
        - biases: biases of the hidden layer
        - input_LB: lower bound of the inputs to the hidden layer
        - input_UB: upper bound of the inputs to the hidden layer
        - num_in_pixels: number of inputs to the input layer
    
    OUTPUT:
        - output_LB: lower bound of the outputs from hidden layer
        - output_UB: upper bound of the outputs from hidden layer
        - num_out_pixels: number of outputs of hidden layer
    '''
    itv = elina_interval_array_alloc(num_in_pixels)

    ## Populate the interval
    for i in range(num_in_pixels):
        elina_interval_set_double(itv[i], input_LB[i], input_UB[i])

    ## construct input abstraction
    element = elina_abstract0_of_box(man, 0, num_in_pixels, itv)
    elina_interval_array_free(itv, num_in_pixels)
    
    # ------------------------------------------------------------------
    # Handle Affine Layer
    # ------------------------------------------------------------------

    # calculate number of outputs
    num_out_pixels = len(weights)
    
    if verbose:
        print("[Network] Input pixels: " + str(num_in_pixels))
        print("[Network] Shape of weights: " + str(np.shape(weights)))
        print("[Network] Shape of biases: " + str(np.shape(biases)))
        print("[Network] Out pixels: " + str(num_out_pixels))

    # Create number of neurons in the layer and populate it
    # with the number of inputs to each neuron in the layer
    dimadd = elina_dimchange_alloc(0, num_out_pixels)    
    for i in range(num_out_pixels):
        dimadd.contents.dim[i] = num_in_pixels

    # Add dimensions to an ElinaAbstract0 pointer i.e. element
    elina_abstract0_add_dimensions(man, True, element, dimadd, False)
    elina_dimchange_free(dimadd)

    # Create the linear expression associated each neuron
    var = num_in_pixels
    for i in range(num_out_pixels):
        tdim = ElinaDim(var)
        linexpr0 = generate_linexpr0(weights[i], biases[i], num_in_pixels)
        # Parallel assignment of several dimensions of an ElinaAbstract0 by using an ElinaLinexpr0Array
        element = elina_abstract0_assign_linexpr_array(man, True, element, tdim, linexpr0, 1, None)
        var += 1

    # Pointer to which semantics we want to follow.
    dimrem = elina_dimchange_alloc(0, num_in_pixels)
    for i in range(num_in_pixels):
        dimrem.contents.dim[i] = i
        
    # Remove dimensions from an ElinaAbstract0
    elina_abstract0_remove_dimensions(man, True, element, dimrem)
    elina_dimchange_free(dimrem)
    
    # get bounds for each output neuron
    bounds = elina_abstract0_to_box(man,element)
    
    output_LB = np.zeros((num_out_pixels, 1), float)
    output_UB = np.zeros((num_out_pixels, 1), float)
    for j in range(num_out_pixels):
        output_LB[j] = bounds[j].contents.inf.contents.val.dbl
        output_UB[j] = bounds[j].contents.sup.contents.val.dbl    
    
    # free out the memory allocations
    elina_interval_array_free(bounds, num_out_pixels)
    elina_abstract0_free(man, element)
    
    return output_LB, output_UB, num_out_pixels

# Initialize the problem variables

In [17]:
!ls /home/riai2018/mnist_nets/

mnist_relu_3_10.txt    mnist_relu_6_100.txt  mnist_relu_9_100.txt
mnist_relu_3_20.txt    mnist_relu_6_200.txt  mnist_relu_9_200.txt
mnist_relu_3_50.txt    mnist_relu_6_20.txt
mnist_relu_4_1024.txt  mnist_relu_6_50.txt


In [18]:
# netname = '/home/riai2018/mnist_nets/mnist_relu_3_10.txt'
# specname = '/home/riai2018/mnist_images/img2.txt'
# epsilon = 0.01072 NOT verified
# epsilon = 0.01071 verified

netname = '/home/riai2018/mnist_nets/mnist_relu_3_50.txt'
specname = '/home/riai2018/mnist_images/img2.txt'
epsilon = 0.01072

In [19]:
with open(netname, 'r') as netfile:
    netstring = netfile.read()
with open(specname, 'r') as specfile:
    specstring = specfile.read()
nn = parse_net(netstring)
x0_low, x0_high = parse_spec(specstring)
LB_N0, UB_N0 = get_perturbed_image(x0_low,0)

In [20]:
numlayer = nn.numlayer 

for layerno in range(numlayer):
    if(nn.layertypes[layerno] in ['ReLU', 'Affine']):
        print(nn.layertypes[layerno])

ReLU
ReLU
ReLU


# Get perturbed label (provided prediction for unperturbed is true)

In [21]:
label, _ = analyze(nn,LB_N0,UB_N0,0) # Get label of unperturbed image, i.e. eps=0
print("Test label: " + str(label))

if(label == int(x0_low[0])):
    LB_N0, UB_N0 = get_perturbed_image(x0_low,epsilon)
else:
    print("image not correctly classified by the network. expected label ",int(x0_low[0]), " classified label: ", label)

Test label: 1


# Start Verification

## 1) Define element for the input 

In [22]:
num_pixels = len(LB_N0)
numlayer = nn.numlayer 
man = elina_box_manager_alloc()

## 2) Iterate over each layer in the network and define the neural network function

In [23]:
nn.ffn_counter = 0
input_LB = copy.deepcopy(LB_N0)
input_UB = copy.deepcopy(UB_N0)
num_in_pixels = num_pixels

use_box = True
verbose = False
LB_list = [input_LB]
UB_list = [input_UB]

for layerno in range(numlayer):
    print("Layer Number: " + str(layerno))
    if(nn.layertypes[layerno] in ['ReLU', 'Affine']):
        if verbose:
            print("Layer Type: %s" % nn.layertypes[layerno])
        # read the layer weights and biases
        weights = nn.weights[nn.ffn_counter]
        biases = nn.biases[nn.ffn_counter]
        np.ascontiguousarray(weights, dtype=np.double)
        np.ascontiguousarray(biases, dtype=np.double)
        
        # only hidden layer
        if (layerno == 0):
            print("HIDDEN!")
            output_LB, output_UB, num_out_pixels = get_hidden_bounds_using_box(man, weights, biases, input_LB, input_UB, num_in_pixels, verbose)
        # ReLU + hidden layer
        else:
            print("RELU + HIDDEN!")
            if use_box:
                output_LB, output_UB, num_out_pixels = get_relu_hidden_bounds_using_box(man, weights, biases, input_LB, input_UB, num_in_pixels, verbose)

        # Iterate to next layer
        input_LB = output_LB
        input_UB = output_UB
        LB_list.append(input_LB)
        UB_list.append(input_UB)

        num_in_pixels = num_out_pixels        
        nn.ffn_counter+=1 
        
        # only ReLU layer
        if(layerno + 1 == numlayer and not
            nn.layertypes[layerno] == "Affine"):
            if verbose:
                print("[OUTPUT] Bounds: ")
                output_LB, output_UB  = output_LB.squeeze(), output_UB.squeeze()
                pprint(np.stack((output_LB, output_UB), axis=1))
            print('---------------')
            print("RELU!")
            output_LB, output_UB, num_out_pixels = get_relu_bounds_using_box(man, input_LB, input_UB, num_in_pixels)
            LB_list.append(output_LB)
            UB_list.append(output_UB)

        if verbose:
            print("[OUTPUT] Bounds: ")
            output_LB, output_UB  = output_LB.squeeze(), output_UB.squeeze()
            pprint(np.stack((output_LB, output_UB), axis=1))
            
        print('---------------')

    else:
        print(' net type not supported')

Layer Number: 0
HIDDEN!
---------------
Layer Number: 1
RELU + HIDDEN!
---------------
Layer Number: 2
RELU + HIDDEN!
---------------
RELU!
---------------


In [24]:
print("[OUTPUT] Bounds: ")
pprint(np.concatenate([LB_list[4], UB_list[4]], axis = 1))

[OUTPUT] Bounds: 
array([[-0.        ,  1.10195037],
       [-0.        , 13.2511222 ],
       [-0.        ,  8.37718904],
       [-0.        ,  6.39543007],
       [-0.        ,  6.46206566],
       [-0.        ,  5.98061365],
       [-0.        ,  6.13388701],
       [-0.        ,  7.35724822],
       [-0.        ,  6.41830464],
       [-0.        ,  6.03332542]])


## Now we have the list of the bounds. Forwards propagate it!

In [25]:
def relu_activations_checker(LBs, UBs):
    """
    This function computes ReLU.
    INPUT
        - LBs: list of lower bound of inputs to all the layers
        - UBs: list of upper bound of inputs to all the layers
    OUTPUT:
        - neuron_states: neuron actiavtion information    
    """
    
    neuron_states = copy.deepcopy(UBs)
    
    output_LB = np.zeros((num_in_pixels), float)
    output_UB = np.zeros((num_in_pixels), float)
    lamda_linear = np.zeros((num_in_pixels), float)
    mu_linear = np.zeros((num_in_pixels), float)
    nontriv_relu = np.zeros((num_in_pixels), bool)
    
    for i in range(len(UBs)):
        for j in range(len(UBs[i])):
            u = UBs[i][j]
            l = LBs[i][j]
            if u <= 0:
                neuron_states[i][j] = -1
            elif l >= 0:
                neuron_states[i][j] = 1
            else:
                neuron_states[i][j] = 0

    return neuron_states

In [48]:
elina_manager_free(man)

Ws = nn.weights
bs = nn.biases
p = 1
nlayer = numlayer

LBs = copy.deepcopy(LB_list)
UBs = copy.deepcopy(UB_list)

neuron_states = relu_activations_checker(LBs, UBs)

In [49]:
import gurobipy as grb

In [50]:
# storing upper and lower bounds for last layer
UB = np.empty_like(bs[-1])
LB = np.empty_like(bs[-1])

# neuron_state is an array: neurons never activated set to -1, neurons always activated set to +1, indefinite set to 0    
# indices
alphas = []
# for n layer network, we have n-1 layers of relu
for i in range(nlayer-1):
    idx_unsure = (neuron_states[i] == 0).nonzero()[0]
    # neuron_state is an integer array for efficiency reasons. We should convert it to float
    alpha = neuron_states[i].astype(np.float32)
    for j in idx_unsure:
        alpha[j] = UBs[i+1][j]/(UBs[i+1][j]-LBs[i+1][j])
    alphas.append(alpha)

In [51]:
start = time.time()

m = grb.Model("LP")
m.setParam("outputflag",0)
# disable parallel Gurobi solver, using 1 thread only
m.setParam("Method",1) # dual simplex
m.setParam("Threads", 1) # only 1 thread
# z and zh are list of lists, each list for one layer of variables
# z starts from 1, matching Zico's notation
z = []
z.append(None)
# z hat starts from 2
zh = []
zh.append(None)
zh.append(None)

if p == "2" or p == "1":
    # ztrans (transformation of z1 only for lp norm), starts from 1 matching z
    ztrans = []
    ztrans.append(None)

## LP codes: 

# we start our label from 1 to nlayer+1 (the last one is the final objective layer)
# valid range for z: 1 to nlayer (z_1 is just input, z_{nlayer} is the last relu layer output)
# valid range for z_hat: 2 to nlayer+1 (there is no z_hat_1 as it is the input, z_{nlayer+1} is final output)
for i in range(1,nlayer+2):
    if i == 1: # first layer
        # first layer, only z exists, no z hat
        zzs = []
        zzts = []
        # UBs[0] is for input x. Create a variable for each input
        # and set its lower and upper bounds
        for j in range(1,len(UBs[0])+1):
            zij = m.addVar(vtype=grb.GRB.CONTINUOUS, lb=LBs[0][j-1], ub=UBs[0][j-1], name="z_"+str(i)+"_"+str(j))
            zzs.append(zij)
            if p == "2" or p == "1":                
                # transformation variable at z1 only
                if p == "2":
                    ztij = m.addVar(vtype=grb.GRB.CONTINUOUS, name="zt_"+str(i)+"_"+str(j))
                elif p == "1":
                    ztij = m.addVar(vtype=grb.GRB.CONTINUOUS, lb=0, name="zt_"+str(i)+"_"+str(j))
                zzts.append(ztij)  
        z.append(zzs)
        if p == "2" or p == "1":
            ztrans.append(zzts)
    elif i< nlayer+1:
        # middle layer, has both z and z hat
        zzs = []
        zzhs = []
        for j in range(1,len(UBs[i-1])+1):
            zij = m.addVar(vtype=grb.GRB.CONTINUOUS, name="z_"+str(i)+"_"+str(j))
            zzs.append(zij)

            zhij = m.addVar(vtype=grb.GRB.CONTINUOUS,lb=-np.inf,name="zh_"+str(i)+"_"+str(j))
            zzhs.append(zhij)
        z.append(zzs)
        zh.append(zzhs)
    else: # last layer, i == nlayer + 1
        # only has z hat, length is the same as the output
        # there is no relu, so no z
        zzhs = []
        for j in range(1,len(bs[-1])+1):
            zhij = m.addVar(vtype=grb.GRB.CONTINUOUS,lb=-np.inf,name="zh_"+str(i)+"_"+str(j))
            zzhs.append(zhij)
        zh.append(zzhs)

m.update()

In [52]:
# Adding weights constraints for all layers
for i in range(1,nlayer+1):
    W = Ws[i-1] # weights of layer i
    for j in range(W.shape[0]):
        """
        sum_term = bs[i-1][j]
        for s in range(W.shape[1]):
            # z start from 1
            sum_term += z[i][s]*W[j,s]
        """
        sum_term = grb.LinExpr(W[j], z[i]) + bs[i-1][j]
        # this is the output of layer i, and let z_hat_{i+1} equal to it
        # z_hat_{nlayer+1} is the final output (logits)
        m.addConstr(sum_term == zh[i+1][j], "weights==_"+str(i)+"_"+str(j))
        # m.addConstr(sum_term <= zh[i+1][j], "weights<=_"+str(i)+"_"+str(j))
        # m.addConstr(sum_term >= zh[i+1][j], "weights>=_"+str(i)+"_"+str(j))

# nlayer network only has nlayer - 1 activations
for i in range(1, nlayer):
    # UBs[0] is the bounds for input x, so start from 1
    for j in range(len(UBs[i])):
        # neuron_states starts from 0
        if neuron_states[i-1][j] == 1:
            m.addConstr(z[i+1][j] == zh[i+1][j], "LPposr==_"+str(j))
            # m.addConstr(z[i+1][j] <= zh[i+1][j], "LPpos<=_"+str(j))
            # m.addConstr(z[i+1][j] >= zh[i+1][j], "LPpos>=_"+str(j))
        elif neuron_states[i-1][j] == -1:
            m.addConstr(z[i+1][j] == 0, "LPneg==_"+str(j))
            # m.addConstr(z[i+1][j] <= 0, "LPneg<=_"+str(j))
            # m.addConstr(z[i+1][j] >= 0, "LPneg>=_"+str(j))
        elif neuron_states[i-1][j] == 0:
            # m.addConstr(z[i+1][j] >= 0, "LPunsure>=0_"+str(j))
            m.addConstr(z[i+1][j] >= zh[i+1][j], "LPunsure>=_"+str(j))
            m.addConstr(z[i+1][j] <= np.asscalar(alphas[i-1][j])*(zh[i+1][j]-np.asscalar(LBs[i][j])), "LPunsure<=_"+str(j))
        else:
            raise(RuntimeError("unknown neuron_state: "+neuron_states[i])) 


#    #finally, add constraints for z[1], the input -> For p == "i", this is already added in the input variable range zij
#    for i in range(len(UBs[0])):
#         m.addConstr(z[1][i] <= UBs[0][i], "inputs+_"+str(i))
#         m.addConstr(z[1][i] >= LBs[0][i], "inputs-_"+str(i))         

if p == "2": 
    #finally, add constraints for z[1] and ztrans[1], the input
    for i in range(len(UBs[0])):
        m.addConstr(ztrans[1][i] == z[1][i] - x0[i], "INPUTtrans_"+str(i))
    # quadratic constraints
    m.addConstr(grb.quicksum(ztrans[1][i]*ztrans[1][i] for i in range(len(UBs[0]))) <= eps*eps, "INPUT L2 norm QCP")
elif p == "1":
     #finally, add constraints for z[1] and ztrans[1], the input
    temp = []
    for i in range(len(UBs[0])):
        tempi = m.addVar(vtype=grb.GRB.CONTINUOUS)
        temp.append(tempi)

    for i in range(len(UBs[0])):
        # absolute constraints: seem that option1 and 2a, 2c are the right answer (compared to p = 2 result) 
        # option 1
        #m.addConstr(ztrans[1][i] >= z[1][i] - x0[i], "INPUTtransPOS_"+str(i))
        #m.addConstr(ztrans[1][i] >= -z[1][i] + x0[i], "INPUTtransNEG_"+str(i))

        # option 2a: same answer as option 1
        # note if we write , the result is different
        #zzz = m.addVar(vtype=grb.GRB.CONTINUOUS)
        #m.addConstr(zzz == z[1][i]-x0[i])
        #m.addConstr(ztrans[1][i] == grb.abs_(zzz), "INPUTtransABS_"+str(i))

        # option 2b: gives different sol as 2a and 2c, guess it's because abs_() has to take a variable,
        # and that's why 2a and 2c use additional variable zzz or temp
        # but now it gives Attribute error on "gurobipy.LinExpr", so can't use this anymore
        #m.addConstr(ztrans[1][i] == grb.abs_(z[1][i]-x0[i]), "INPUTtransABS_"+str(i))

        # option 2c: same answer as 2a
        m.addConstr(temp[i] == z[1][i]-x0[i])
        m.addConstr(ztrans[1][i] == grb.abs_(temp[i]), "INPUTtransABS_"+str(i))    

        # option 3: same answer as 2b
        #m.addConstr(ztrans[1][i] <= z[1][i] - x0[i], "INPUTtransPOS_"+str(i))
        #m.addConstr(ztrans[1][i] >= -z[1][i] + x0[i], "INPUTtransNEG_"+str(i))


    # L1 constraints
    m.addConstr(grb.quicksum(ztrans[1][i] for i in range(len(UBs[0]))) <= eps, "INPUT L1 norm")



# another way to write quadratic constraints
###expr = grb.QuadExpr()
###expr.addTerms(np.ones(len(UBs[0])), z[1], z[1])
###m.addConstr(expr <= eps*eps)

m.update()

print("[L2][LP solver initialized] time_lp_init = {:.4f}".format(time.time() - start))
# for middle layers, need to compute full bounds

# compute upper bounds        
# z_hat_{nlayer+1} is the logits (final output, or inputs for layer nlayer+1)
##for j in [pred_label,target_label]:
for j in range(Ws[nlayer-1].shape[0]):
    m.setObjective(zh[nlayer+1][j], grb.GRB.MAXIMIZE)    
    # m.write('grbtest_LP_2layer_'+str(j)+'.lp')    
    start = time.time()
    m.optimize()
    UB[j] = m.objVal
    m.reset()
    print("[L2][upper bound solved] j = {}, time_lp_solve = {:.4f}".format(j, time.time() - start))

# compute lower bounds        
##for j in [pred_label,target_label]:
for j in range(Ws[nlayer-1].shape[0]):
    m.setObjective(zh[nlayer+1][j], grb.GRB.MINIMIZE)    
    # m.write('grbtest_LP_2layer_'+str(j)+'.lp')    
    start = time.time()
    m.optimize()
    LB[j] = m.objVal
    m.reset()
    print("[L2][lower bound solved] j = {}, time_lp_solve = {:.4f}".format(j, time.time() - start))

[L2][LP solver initialized] time_lp_init = 5.8233


AttributeError: b"Unable to retrieve attribute 'objVal'"

In [46]:
LB[LB < 0] = 0
print("[OUTPUT] Bounds: ")
pprint(np.stack([LB, UB], axis = 1))

[OUTPUT] Bounds: 
array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])


In [36]:
# if epsilon is zero, try to classify else verify robustness 
verified_flag = True
predicted_label = 2
if(LB_N0[0]==UB_N0[0]):
    for i in range(num_out_pixels):
        inf = LB[i]
        flag = True
        for j in range(num_out_pixels):
            if(j!=i):
                sup = UB[j]
                if(inf<=sup):
                    flag = False
                    break
        if(flag):
            predicted_label = i
            break    
else:
    inf = LB[label]
    for j in range(num_out_pixels):
        if(j!=label):
            sup = UB[j]
            if(inf<=sup):
                predicted_label = label
                verified_flag = False
                break

if(verified_flag):
    print("verified")
else:
    print("can not be verified")  

can not be verified
